In [30]:
import torch
import numpy as np

In [31]:
torch.cuda.is_available()

True

In [61]:
import numpy as np

def cos_sim(x, y):
    numerator = np.dot(x, y)
    x_normalised = np.sqrt(np.sum(x**2))
    y_normalised = np.sqrt(np.sum(y**2))
    denominator = x_normalised * y_normalised
    cosine_similarity = numerator / denominator
    return cosine_similarity

In [132]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

from transformers import BertTokenizer, BertModel

model = BertModel.from_pretrained("bert-base-uncased",output_hidden_states=True)
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model.eval()




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [145]:
def tokenized_indexed_tensor(text):
    
    #text = "A dog is a dog"
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text=tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])   
    
    return tokens_tensor, segments_tensors



In [225]:
def encode_sentences(sentence):
    tokens_tensor, segement_tensors = tokenized_indexed_tensor(sentence)
    
    outputs = model( tokens_tensor, segement_tensors)
    hidden_states = outputs[2]
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    #print(token_embeddings.size())
    token_vecs = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

    
    
    return sentence_embedding.detach().numpy()

In [226]:

sentences=['this cat is ugly', 'this dog great']
embeddings=[]
for s in sentences:
    embeddings.append(encode_sentences(s))


Our final sentence embedding vector of shape: torch.Size([768])
Our final sentence embedding vector of shape: torch.Size([768])


In [227]:
print(cos_sim(embeddings[0],embeddings[1]))

0.8905075
